In [4]:
import keras.backend as K
import numpy as np
import json
import shap
import tensorflow as tf
import warnings

import os


from PIL import Image

In [15]:
import random

In [5]:
#Cargo el modelo
# Specify the path to your saved model file (.h5 or .hdf5 format)
model_path = 'old_models/jorgenet_private_dataset/modelo_entrenado.h5'
# Load the model
model = tf.keras.models.load_model(model_path)


In [6]:
def pipeline(image_path): 
 
 #preprrocess 
 img = tf.keras.preprocessing.image.load_img(image_path, target_size=(255,255))
 x = tf.keras.preprocessing.image.img_to_array(img)    
 x = np.expand_dims(x, axis=0)

 #Predict
 prediction = model.predict(x)
 predicted_class = np.argmax(prediction)
 return(predicted_class)




In [7]:
dataset_folder = "private_dataset"

categories = os.listdir(dataset_folder)
X = []
y = []

for i, category in enumerate(categories):
    category_folder = os.path.join(dataset_folder, category)
    image_files = os.listdir(category_folder)
    
    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        image = Image.open(image_path).resize((224, 224))  # Resize the image to match the input size of your model
        image = np.array(image)
        X.append(image)
        y.append(i)

X = np.array(X)
y = np.array(y)

In [18]:
# Randomly select indices from X to create the background dataset
background_indices = random.sample(range(len(X)), 20)
background_images = X[background_indices]

# Create a DeepExplainer with the background dataset
e = shap.DeepExplainer(model, background_images)

ValueError: Input 0 of layer dense_9 is incompatible with the layer: expected axis -1 of input shape to have value 25088 but received input with shape (20, 18432)

In [11]:
# explain predictions of the model on three images
e = shap.DeepExplainer(model,background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X[1:3])

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


ValueError: Input 0 of layer dense_9 is incompatible with the layer: expected axis -1 of input shape to have value 25088 but received input with shape (3, 18432)

In [13]:
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 255, 255, 3)       0         
_________________________________________________________________
input_4 (InputLayer)         multiple                  0         
_________________________________________________________________
random_flip_3 (RandomFlip)   (None, 255, 255, 3)       0         
_________________________________________________________________
random_rotation_3 (RandomRot (None, 255, 255, 3)       0         
_________________________________________________________________
random_zoom_3 (RandomZoom)   (None, 255, 255, 3)       0         
_________________________________________________________________
random_contrast_3 (RandomCon (None, 255, 255, 3)       0         
_________________________________________________________________
random_translation_3 (Random (None, 255, 255, 3)      